In [ ]:
#Importando bibliotecas para webscrapping e manipulações iniciais
import bs4 as bs
import requests
import pandas as pd
import csv
import numpy as np

#Importando bibliotecas para tratamento de dados e exploração das informações
import nltk, os, spacy
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import RSLPStemmer
from spacy.lang.pt.stop_words import STOP_WORDS
import unicodedata
import re
from nltk.stem import SnowballStemmer
import nltk
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
from pandas_profiling import ProfileReport

#importando bibliotecas de machine learning
import pickle
import time
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.metrics import confusion_matrix
import pickle
import time
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#Lendo a primeira página
df= pd.read_html('http://normas.receita.fazenda.gov.br/sijut2consulta/consulta.action?facetsExistentes=tipoAtoFacet%2CsiglaOrgaoFacet&orgaosSelecionados=&tiposAtosSelecionados=72&lblTiposAtosSelecionados=SC&ordemColuna=&ordemDirecao=&tipoConsulta=formulario&tipoAtoFacet=Solu%C3%A7%C3%A3o+de+Consulta&siglaOrgaoFacet=Cosit&anoAtoFacet=&termoBusca=&numero_ato=&tipoData=2&dt_inicio=&dt_fim=&ano_ato=&optOrdem=relevancia&p=1',
                 attrs = {'id': "tabelaAtos"}, 
                 skiprows = 1, 
                 header = 0, 
                 flavor = 'bs4')[0][:-2]


In [ ]:
#Lendo as demais páginas e fazendo append no DF criado
Base_url ='http://normas.receita.fazenda.gov.br/sijut2consulta/consulta.action?facetsExistentes=tipoAtoFacet%2CsiglaOrgaoFacet&orgaosSelecionados=&tiposAtosSelecionados=72&lblTiposAtosSelecionados=SC&ordemColuna=&ordemDirecao=&tipoConsulta=formulario&tipoAtoFacet=Solu%C3%A7%C3%A3o+de+Consulta&siglaOrgaoFacet=Cosit&anoAtoFacet=&termoBusca=&numero_ato=&tipoData=2&dt_inicio=&dt_fim=&ano_ato=&optOrdem=relevancia&p={}'
for pg in range(2,48):
    df2 = pd.read_html(Base_url.format(pg), 
                       attrs = {'id': "tabelaAtos"}, 
                       skiprows = 1, 
                       header = 0, 
                       flavor = 'bs4')[0][:-2]
    df = df.append(df2, ignore_index = True, sort= False)


In [ ]:
#Visualizando o Data Frame
df
     

In [ ]:
#Cópia de backup
df2 = df.copy()

In [ ]:
df2.to_csv('dataframe_baixado_backup', sep = ';', encoding = 'utf-8-sig') 
#Último download. Utilizar o arquivo caso tenha que rodar código novamente

In [ ]:
df = pd.read_csv('dataframe_baixado_backup', sep = ';', encoding = 'utf-8-sig') 

In [ ]:
df.info()

In [ ]:
df= df2

In [ ]:
lista_emen = df["Ementa"]
lista_emen[1]

In [ ]:
#Colocando as ementas em uma lista e convertendo para uppercase
lista_emen = df["Ementa"]
lista_ementa = []

for i in lista_emen:
    a= i.upper()
    lista_ementa.append(a)

lista_ementa

In [ ]:
len(lista_ementa)

In [ ]:
#Criando a coluna assuntos - Classe

a1 = 'Assunto: Classificação de Mercadorias'
a2 = 'Assunto: Simples Nacional'
a3 = 'Assunto: CONTRIBUIÇÃO PARA O FIN'
a4 = 'Assunto: Imposto sobre a Renda de Pessoa Jurídica'
a5 = 'Assunto: Obrigações Acessórias'
a6 = 'Assunto: IMPOSTO SOBRE A RENDA RETIDO NA FONTE'
a7 = 'Assunto: Contribuição para o PIS/Pasep'
a8 = 'Assunto: Imposto sobre Operações de Crédito, Câmbio e Seguros ou relativas a Títulos ou Valores Mobiliários - IOF'
a9 = 'Assunto: Contribuições Sociais Previdenciárias'
a10 = 'Assunto: Imposto sobre Produtos Industrializados'
a11 = 'Assunto: Imposto sobre a Importação'
a12 = 'Assunto: REGIMES ADUANEIROS'
a13 = 'Assunto: NORMAS GERAIS DE DIREITO TRIBUTÁRIO'
a14 = 'Assunto: NORMAS DE ADMINISTRAÇÃO TRIBUTÁRIA'
a15 = 'Assunto: Imposto sobre Operações de Crédito, Câmbio e Seguros ou relativas a Títulos ou Valores Mobiliários – IOF'
a16 = 'Assunto: Imposto sobre a Renda de Pessoa Física'
a17 = 'Assunto: Normas Gerais'
a18 = 'Assunto: Normas de Administração Tributária'
a19 = 'Assunto: PESSOA JURÍDICA'
a20 = 'Assunto: PESSOA FÍSICA'
a21 = 'Assunto: OBRIGAÇÕES ACESSÓRIAS'
a22 = 'Assunto: IMPOSTO SOBRE A RENDA RETIDO NA FONTE'
a23 = 'Assunto: CONTRIBUIÇÃO DE INTERVENÇÃO NO DOMÍNIO ECONÔMICO'
a24 = 'Assunto: IMPOSTO SOBRE A PROPRIEDADE TERRITORIAL RURAL'
a25 = 'Assunto: SEGURIDADE SOCIAL'
a26 = 'Retificação'
a27 = 'Assunto: CONTRIBUIÇÃO SOCIAL SOBRE O LUCRO LÍQUIDO'
a28 = 'Assunto: CONTRIBUIÇÕES PREVIDENCIÁRIAS'
a29 = 'Assunto: PROCESSO ADMINISTRATIVO FISCAL'
a30 = 'Assunto: Contribuições para a Previdência Social'
a31 = 'Assunto: REGIME ESPECIAL DE REGULARIZAÇÃO CAMBIAL E TRIBUTÁRIA'
a32 = 'Assunto: Outros Tributos ou Contribuições'
a33 = 'Assunto: REGIME ESPECIAL DE REGULARIZAÇÃO'
a34 = 'Assunto: DIREITOS ANTIDUMPING'
a35 = 'Assunto: IMPOSTO DE RENDA RETIDO NA FONTE'
a36 = 'ASSUNTO: IMPOSTO SOBRE A RENDA DA PESSOA JURÍDICA '
a37 = 'ASSUNTO: Classificação de Mercadorias'
a38 = 'Assunto:CONTRIBUIÇÃO PARA O FIN'
a39 = 'ASSUNTO:IMPOSTO SOBRE A RENDA DE PESSOA JURÍDICA'
a40 = 'ASSUNTO:Contribuição para o PIS/Pasep'
a41 = 'Assunto:Imposto sobre a Renda Retido na Fonte'
a42 = 'Contribuições Sociais e Previdenciárias'
a43 = 'Normas Gerais de Direito Tributário'
a44 = 'IOF'

classe_assunto=[]


for i in lista_ementa:
    if i.startswith(a1.upper()) == True:
        classe_assunto.append(a1)
    elif i.startswith(a2.upper())==True:
        classe_assunto.append(a2)
    elif i.startswith(a3.upper())==True:
         classe_assunto.append(a42)
    elif i.startswith(a4.upper())==True:
        classe_assunto.append(a4)
    elif i.startswith(a5.upper())==True:
        classe_assunto.append(a5)
    elif i.startswith(a6.upper())==True:
        classe_assunto.append(a6)
    elif i.startswith(a7.upper())== True:
         classe_assunto.append(a42)
    elif i.startswith(a8.upper())== True:
        classe_assunto.append(a44)
    elif i.startswith(a9.upper())== True:
        classe_assunto.append(a42)
    elif i.startswith(a10.upper())== True:
        classe_assunto.append(a10)
    elif i.startswith(a11.upper())== True:
         classe_assunto.append(a11)
    elif i.startswith(a12.upper())== True:
        classe_assunto.append(a12)
    elif i.startswith(a13.upper())== True:
        classe_assunto.append(a43)
    elif i.startswith(a14.upper())== True:
        classe_assunto.append(a14)
    elif i.startswith(a15.upper())== True:
        classe_assunto.append(a44)
    elif i.startswith(a16.upper())== True:
        classe_assunto.append(a16)
    elif i.startswith(a17.upper())== True:
        classe_assunto.append(a43)
    elif i.startswith(a18.upper())== True:
        classe_assunto.append(a18)
    elif i.startswith(a19.upper())== True:
        classe_assunto.append(a4)
    elif i.startswith(a20.upper())== True:
        classe_assunto.append(a16)
    elif i.startswith(a21.upper())== True:
        classe_assunto.append(a21)
    elif i.startswith(a22.upper())== True:
        classe_assunto.append(a6)
    elif i.startswith(a23.upper())== True:
        classe_assunto.append(a23)
    elif i.startswith(a24.upper())== True:
        classe_assunto.append(a24)
    elif i.startswith(a25.upper())== True:
        classe_assunto.append(a42)      
    elif i.startswith(a26.upper())== True:
        classe_assunto.append(a26)
    elif i.startswith(a27.upper())== True:
        classe_assunto.append(a42)
    elif i.startswith(a28.upper())== True:
        classe_assunto.append(a42)
    elif i.startswith(a29.upper())== True:
        classe_assunto.append(a29)      
    elif i.startswith(a30.upper())== True:
        classe_assunto.append(a42)
    elif i.startswith(a31.upper())== True:
        classe_assunto.append(a31)
    elif i.startswith(a32.upper())== True:
        classe_assunto.append(a32)
    elif i.startswith(a33.upper())== True:
        classe_assunto.append(a33)
    elif i.startswith(a34.upper())== True:
        classe_assunto.append(a34)
    elif i.startswith(a35.upper())== True:
        classe_assunto.append(a6)
    elif i.startswith(a36.upper())== True:
        classe_assunto.append(a4)
    elif i.startswith(a37.upper())== True:
        classe_assunto.append(a1)
    elif i.startswith(a38.upper())==True:
         classe_assunto.append(a42)
    elif i.startswith(a39.upper())==True:
         classe_assunto.append(a4)
    elif i.startswith(a40.upper())==True:
         classe_assunto.append(a42)
    elif i.startswith(a41.upper())==True:
        classe_assunto.append(a6)
    else:
        classe_assunto.append("Não encontrado")

classe_assunto.count("Não encontrado")
    


In [ ]:
classe_assunto
    

In [ ]:
#Excluindo a parte "assunto" do assunto

B1 = "ASSUNTO: "
x = classe_assunto
y = []
assunto_organizado = []

for i in x:
    h = i.upper()
    y.append(h)

for i in y:
    j = i.replace(B1.upper(), "") 
    assunto_organizado.append(j)

In [ ]:
#Excluindo os assuntos da ementa

a1 = 'Assunto: Classificação de Mercadorias'
a2 = 'Assunto: Simples Nacional'
a3 = 'Assunto: CONTRIBUIÇÃO PARA O FIN'
a4 = 'Assunto: Imposto sobre a Renda de Pessoa Jurídica'
a5 = 'Assunto: Obrigações Acessórias'
a6 = 'Assunto: IMPOSTO SOBRE A RENDA RETIDO NA FONTE'
a7 = 'Assunto: Contribuição para o PIS/Pasep'
a8 = 'Assunto: Imposto sobre Operações de Crédito, Câmbio e Seguros ou relativas a Títulos ou Valores Mobiliários - IOF'
a9 = 'Assunto: Contribuições Sociais Previdenciárias'
a10 = 'Assunto: Imposto sobre Produtos Industrializados'
a11 = 'Assunto: Imposto sobre a Importação'
a12 = 'Assunto: REGIMES ADUANEIROS'
a13 = 'Assunto: NORMAS GERAIS DE DIREITO TRIBUTÁRIO'
a14 = 'Assunto: NORMAS DE ADMINISTRAÇÃO TRIBUTÁRIA'
a15 = 'Assunto: Imposto sobre Operações de Crédito, Câmbio e Seguros ou relativas a Títulos ou Valores Mobiliários – IOF'
a16 = 'Assunto: Imposto sobre a Renda de Pessoa Física'
a17 = 'Assunto: Normas Gerais'
a18 = 'Assunto: Normas de Administração Tributária'
a19 = 'Assunto: PESSOA JURÍDICA'
a20 = 'Assunto: PESSOA FÍSICA'
a21 = 'Assunto: OBRIGAÇÕES ACESSÓRIAS'
a22 = 'Assunto: IMPOSTO SOBRE A RENDA RETIDO NA FONTE'
a23 = 'Assunto: CONTRIBUIÇÃO DE INTERVENÇÃO NO DOMÍNIO ECONÔMICO'
a24 = 'Assunto: IMPOSTO SOBRE A PROPRIEDADE TERRITORIAL RURAL'
a25 = 'Assunto: SEGURIDADE SOCIAL'
a26 = 'Retificação'
a27 = 'Assunto: CONTRIBUIÇÃO SOCIAL SOBRE O LUCRO LÍQUIDO'
a28 = 'Assunto: CONTRIBUIÇÕES PREVIDENCIÁRIAS'
a29 = 'Assunto: PROCESSO ADMINISTRATIVO FISCAL'
a30 = 'Assunto: Contribuições para a Previdência Social'
a31 = 'Assunto: REGIME ESPECIAL DE REGULARIZAÇÃO CAMBIAL E TRIBUTÁRIA'
a32 = 'Assunto: Outros Tributos ou Contribuições'
a33 = 'Assunto: REGIME ESPECIAL DE REGULARIZAÇÃO'
a34 = 'Assunto: DIREITOS ANTIDUMPING'
a35 = 'Assunto: IMPOSTO DE RENDA RETIDO NA FONTE'
a36 = 'ASSUNTO: IMPOSTO SOBRE A RENDA DA PESSOA JURÍDICA '
a37 = 'ASSUNTO: Classificação de Mercadorias'
a38 = 'Assunto:CONTRIBUIÇÃO PARA O FIN'
a39 = 'ASSUNTO:IMPOSTO SOBRE A RENDA DE PESSOA JURÍDICA'
a40 = 'ASSUNTO:Contribuição para o PIS/Pasep'
a41 = 'Assunto:Imposto sobre a Renda Retido na Fonte'

ementa_organizada=[]


for i in lista_ementa:
    if i.startswith(a1.upper()) == True:
        a = i.replace(a1.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a2.upper())==True:
        a = i.replace(a2.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a3.upper())==True:
        a = i.replace(a3.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a4.upper())==True:
        a = i.replace(a4.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a5.upper())==True:
        a = i.replace(a5.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a6.upper())==True:
        a = i.replace(a6.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a7.upper())== True:
        a = i.replace(a7.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a8.upper())== True:
        a = i.replace(a8.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a9.upper())== True:
        a = i.replace(a9.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a10.upper())== True:
        a = i.replace(a10.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a11.upper())== True:
        a = i.replace(a11.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a12.upper())== True:
        a = i.replace(a12.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a13.upper())== True:
        a = i.replace(a13.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a14.upper())== True:
        a = i.replace(a14.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a15.upper())== True:
        a = i.replace(a15.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a16.upper())== True:
        a = i.replace(a16.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a17.upper())== True:
        a = i.replace(a17.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a18.upper())== True:
        a = i.replace(a18.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a19.upper())== True:
        a = i.replace(a19.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a20.upper())== True:
        a = i.replace(a20.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a21.upper())== True:
        a = i.replace(a21.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a22.upper())== True:
        a = i.replace(a22.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a23.upper())== True:
        a = i.replace(a23.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a24.upper())== True:
        a = i.replace(a24.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a25.upper())== True:
        a = i.replace(a25.upper(), "") 
        ementa_organizada.append(a)      
    elif i.startswith(a26.upper())== True:
        a = i.replace(a26.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a27.upper())== True:
        a = i.replace(a27.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a28.upper())== True:
        a = i.replace(a28.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a29.upper())== True:
        a = i.replace(a29.upper(), "") 
        ementa_organizada.append(a)      
    elif i.startswith(a30.upper())== True:
        a = i.replace(a30.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a31.upper())== True:
        a = i.replace(a31.upper(), "")  
        ementa_organizada.append(a)
    elif i.startswith(a32.upper())== True:
        a = i.replace(a32.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a33.upper())== True:
        a = i.replace(a33.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a34.upper())== True:
        a = i.replace(a34.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a35.upper())== True:
        a = i.replace(a35.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a36.upper())== True:
        a = i.replace(a36.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a37.upper())== True:
        a = i.replace(a37.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a38.upper())==True:
        a = i.replace(a38.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a39.upper())==True:
        a = i.replace(a39.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a40.upper())==True:
        a = i.replace(a40.upper(), "") 
        ementa_organizada.append(a)
    elif i.startswith(a41.upper())==True:
        a = i.replace(a41.upper(), "") 
        ementa_organizada.append(a)
    else:
        ementa_organizada.append("Não encontrado")
        
ementa_organizada.count("Não encontrado")

In [ ]:
ementa_organizada

In [ ]:
assunto_organizado

In [ ]:
#Atualizando o dataset com as colunas já formatadas

df["Ementa"] = ementa_organizada
df["Assunto"] = assunto_organizado
df = df.drop(columns=['Unnamed: 5'])
df = df.drop(columns=['Unnamed: 0'])
df

In [ ]:
#Fazendo backup do df em csv
df.to_csv("df3-backup.csv", sep = ';', encoding = 'utf-8-sig')

In [ ]:
#Construindo a Pivot Table1
table1 = pd.pivot_table(df, values='Nº do ato', index=['Assunto'], aggfunc='count' )

In [ ]:
len(table1)

In [ ]:
df = df.query('assunto != ["OUTROS TRIBUTOS OU CONTRIBUIÇÕES","DIREITOS ANTIDUMPING","REGIME ESPECIAL DE REGULARIZAÇÃO CAMBIAL E TRIBUTÁRIA","RETIFICAÇÃO", "PROCESSO ADMINISTRATIVO FISCAL", "NORMAS GERAIS", "IMPOSTO SOBRE A PROPRIEDADE TERRITORIAL RURAL","NÃO ENCONTRADO" ]') 


In [ ]:
df

In [ ]:
#Construindo a Pivot Table2
table2 = pd.pivot_table(df, values= 'ementa', index=['assunto'], aggfunc='count' )

In [ ]:
print(table2)
print('-------------------------------------------')
print("Quantidade de classes: " + str(len(table2)))
print("Quantidade de linhas no dataset: " + str(len(df)))

In [ ]:
df.to_csv('df_blackup.csv', sep = ',', encoding = 'utf-8-sig')

In [ ]:
df=pd.read_csv('df_blackup.csv', sep = ',', encoding = 'utf-8-sig')

In [ ]:
df

In [ ]:
df = pd.DataFrame(data={'ementa': df['Ementa'], 'assunto': df['Assunto']})

In [ ]:
df

In [ ]:
df= df.astype({'ementa': 'string',
           'assunto': 'category'})

In [ ]:
df.info()

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

In [ ]:
#SEGUNDA LIMPEZA
def padronizando(df, text_field):
    df[text_field] = df[text_field].str.replace(r"CÓDIGO", "")
    df[text_field] = df[text_field].str.replace(r"EMENTA", "")
    df[text_field] = df[text_field].str.replace(r"ANCIAMENTO DA SEGURIDADE SOCIAL", "")
    df[text_field] = df[text_field].str.lower()
    return df

In [ ]:
df = padronizando(df,'ementa')

In [ ]:
df = pd.DataFrame(data={'ementa': df['Ementa'], 'assunto': df['Assunto']})
df

In [ ]:
#Unindo as stopwords de NLTK e SPACY
stopwordsextras =[]
stopwordsmantidas=[]
stopwordsnltk = nltk.corpus.stopwords.words('portuguese')
stopwordspacy = [x for x in STOP_WORDS]
stopwordsextra = stopwordsnltk + [x for x in stopwordspacy if x not in stopwordsnltk] + [x for x in stopwordsextras if x not in stopwordsnltk]
stopwords = [x for x in stopwordsextra if x not in stopwordsmantidas]


print(stopwords)


In [ ]:
stopwords.remove('não')

In [ ]:
print(stopwords)

In [ ]:
#Pré-Processamento do texto

#retirando acentos
ementa3_clean = [unicodedata.normalize('NFKD', s).encode('ISO-8859-1', 'ignore').decode('ISO-8859-1') for s in df['ementa']]

#retirando todos os caracteres especiais ->[re.sub('[^\w\d\s]',r'
ementa3_clean2 = [re.sub(r'[^a-zA-z]',r' ', doc).lower() for doc in ementa3_clean ]
df['ementa_processada'] = ementa3_clean2


#primeiro tokeniza, depois retirando as sopwords
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
df["tokens"] = df["ementa_processada"].apply(tokenizer.tokenize)


#Removendo as stopwords
def remove_stopwords(tokens):
    return[token for token in tokens if token not in stopwords]
corpus_stops = [' '.join(remove_stopwords(doc)) for doc in df['tokens']]


#Tokenizando novamente a lista limpa das stop words
df['ementa_processada2'] = corpus_stops
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
df["tokens2"] = df["ementa_processada2"].apply(tokenizer.tokenize)


#Realizando o stemming depois de retirada as stopwords
def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

a = df["tokens2"]
ementa_processada3= []

for i in a:
    ementa_processada3.append(Stemming(i))

#colocando com stemming no df
df['ementa_processada3'] = ementa_processada3


In [ ]:
df

In [ ]:
#Pré-Processamento do texto2


#primeiro tokeniza, para depois retirar as sopwords
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
df["tokens"] = df["ementa"].apply(tokenizer.tokenize)


#Removendo as stopwords
def remove_stopwords(tokens):
    return[token for token in tokens if token not in stopwords]
corpus_stops = [' '.join(remove_stopwords(doc)) for doc in df['tokens']]

#retirando acentos
df['ementa_limpa']= corpus_stops
ementa3_clean = [unicodedata.normalize('NFKD', s).encode('ISO-8859-1', 'ignore').decode('ISO-8859-1') for s in df['ementa_limpa']]

#retirando todos os caracteres especiais ->[re.sub('[^\w\d\s]',r'
ementa_limpa2 = [re.sub(r'[^a-zA-z]',r' ', doc).lower() for doc in ementa3_clean ]
df['ementa_processada'] = ementa_limpa2

#Tokenizando novamente a lista limpa das stop words
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
df["tokens2"] = df["ementa_processada"].apply(tokenizer.tokenize)


#Realizando o stemming depois de retirada as stopwords
def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

a = df["tokens2"]
ementa_processada2= []

for i in a:
    ementa_processada2.append(Stemming(i))

#colocando com stemming no df
df['ementa_processada2'] = ementa_processada2


In [ ]:
#destokenizando
from nltk.tokenize.treebank import TreebankWordDetokenizer
ementa_processada3 = []

for i in ementa_processada2:
    ementa_processada3.append(TreebankWordDetokenizer().detokenize(i))
df['ementa_processada3'] = ementa_processada3


In [ ]:
#Primeira comparação
print('Tamanho do corpus sem processar: ' + str(len(df['ementa'][1])))
print('\nExemplo:')
print('\n' + df['ementa'][1])
print('\nTamanho do corpus processado: ' + str(len(df['ementa_processada3'][1])))
print('\nExemplo:')
print('\n'+df['ementa_processada3'][1])

In [ ]:
df

In [ ]:
#Criando o df_limpo

df_limpo = pd.DataFrame(data={'ementa': df['ementa'],
                              'ementa_processada': df['ementa_processada3'],
                              'assunto': df['assunto']})
df_limpo

In [ ]:
df_limpo= df_limpo.astype({'ementa': 'string',
           'assunto': 'category',
            'ementa_processada': 'string'
                  })

In [ ]:
df_limpo.info()

In [ ]:
#Calculando o tamanho individual das ementas apenas para fins de comparação entre elas
tam_ementa_processada = []
tam_ementa_tokenizada= []

for x in df_limpo["ementa_processada"]:
    tam_ementa_processada.append(len(x))
for x in df_limpo["ementa_tokenizada"]:
    tam_ementa_tokenizada.append(len(x))
    
df_limpo['tam_ementa_processada'] = tam_ementa_processada
df_limpo['tam_ementa_tokenizada'] = tam_ementa_tokenizada

In [ ]:
df_limpo

In [ ]:
#Calculando o tamanho individual das ementas apenas para fins de comparação entre elas2
tam_ementa = []
tam_ementa_processada= []

for x in df_limpo["ementa"]:
    tam_ementa.append(len(x))
for x in df_limpo["ementa_processada"]:
    tam_ementa_processada.append(len(x))
    
df_limpo['tam_ementa'] = tam_ementa
df_limpo['tam_ementa_processada'] = tam_ementa_processada

In [ ]:
#Usando o pandas Profiling
profile = ProfileReport(df_limpo, title='Analise Exploratória do Data Frame',html={'style':{'full_width':True}})


In [ ]:
profile.to_notebook_iframe()

In [ ]:
#Salvando a primeira versão do DF Limpo
df_limpo.to_csv('df_limpo.csv', sep = ';', encoding = 'utf-8-sig')

In [ ]:
#Confirmando duplicados
print("Existem Duplicados? : " +  str(any(df_limpo['ementa_processada'].duplicated())))
print("Linhas duplicadas: " +  str(4541 - df_limpo['ementa_processada'].nunique()))

In [ ]:
df_limpo = df_limpo.drop_duplicates('ementa_processada')

In [ ]:
len(df_limpo)

In [ ]:
cor = df_limpo['tam_ementa'].corr(df_limpo['tam_ementa_processada'], method = 'pearson')
cor

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

texto = ' '. join(df_limpo['ementa_processada'])
wordCloud = WordCloud(max_font_size = 160, width = 1400, height = 700, colormap = 'viridis', background_color = 'white').generate(texto)



In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(wordCloud)
plt.axis('off')

In [ ]:
df_limpo.shape

In [ ]:
df_limpo2 = df_limpo.drop_duplicates('ementa')

In [ ]:
print(df_limpo2.shape)
print(df_limpo2['ementa'].nunique())

In [ ]:
novas_stopwords = ['legais', 'dispositivos', 'aprovada', 'decreto', 'rfb', 'rgi','legal', 'dispositivo']
for i in novas_stopwords:
    stopwords.append(i)

In [ ]:
print(stopwords)

In [ ]:
#Pré-Processamento do texto2


#primeiro tokeniza, para depois retirar as sopwords
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
df["tokens"] = df["ementa"].apply(tokenizer.tokenize)


#Removendo as stopwords
def remove_stopwords(tokens):
    return[token for token in tokens if token not in stopwords]
corpus_stops = [' '.join(remove_stopwords(doc)) for doc in df['tokens']]

#retirando acentos
df['ementa_limpa']= corpus_stops
ementa3_clean = [unicodedata.normalize('NFKD', s).encode('ISO-8859-1', 'ignore').decode('ISO-8859-1') for s in df['ementa_limpa']]

#retirando todos os caracteres especiais ->[re.sub('[^\w\d\s]',r'
ementa_limpa2 = [re.sub(r'[^a-zA-z]',r' ', doc).lower() for doc in ementa3_clean ]
df['ementa_processada'] = ementa_limpa2

#Tokenizando novamente a lista limpa das stop words
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
df["tokens2"] = df["ementa_processada"].apply(tokenizer.tokenize)


#Realizando o stemming depois de retirada as stopwords
def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

a = df["tokens2"]
ementa_processada2= []

for i in a:
    ementa_processada2.append(Stemming(i))

#colocando com stemming no df
df['ementa_processada2'] = ementa_processada2


In [ ]:
#destokenizando2
from nltk.tokenize.treebank import TreebankWordDetokenizer
ementa_processada3 = []

for i in ementa_processada2:
    ementa_processada3.append(TreebankWordDetokenizer().detokenize(i))
df['ementa_processada3'] = ementa_processada3

In [ ]:
#Criando o df_limpo2

df_limpo = pd.DataFrame(data={'ementa': df['ementa'],
                              'ementa_processada': df['ementa_processada3'],
                              'assunto': df['assunto']})
df_limpo

In [ ]:
#Confirmando duplicados2
print("Existem Duplicados? : " +  str(any(df_limpo['ementa_processada'].duplicated())))
print("Linhas duplicadas: " +  str(4541 - df_limpo['ementa_processada'].nunique()))

In [ ]:
#retirando os duplicados2
df_limpo = df_limpo.drop_duplicates('ementa_processada')

In [ ]:
len(df_limpo)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

texto = ' '. join(df_limpo['ementa_processada'])
wordCloud = WordCloud(max_font_size = 160, width = 1400, height = 700, colormap = 'viridis', background_color = 'white').generate(texto)



In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(wordCloud)
plt.axis('off')

In [ ]:
table3 = pd.pivot_table(df_limpo, values='ementa_processada', index=['assunto'],  aggfunc='count' )


In [ ]:
df_limpo = df_limpo.query('assunto != ["NÃO ENCONTRADO"]') 


In [ ]:
table3

In [ ]:
df_limpo = df_limpo.drop(columns =['Unnamed: 0'])

In [ ]:
df_limpo.info()

In [ ]:
df_limpo

In [ ]:
df_limpo = df_limpo.dropna()

In [ ]:
list_corpus = df_limpo["ementa_processada"].tolist()

In [ ]:
len(list_corpus)

In [ ]:
#tfidf 
ngram = (1,2)
vetorizador = TfidfVectorizer(ngram_range = ngram ,min_df = 2, max_df = .85, max_features = 5000)
# Separando o texto da variável alvo
list_corpus = df_limpo["ementa_processada"].tolist()
list_labels = df_limpo["assunto"].tolist()

#transformando o corpus em matriz esparsa 
matriz_esparsa_tfdif = vetorizador.fit_transform(list_corpus)


In [ ]:
tokens = vetorizador.get_feature_names()

In [ ]:
matriz_esparsadf = pd.DataFrame(matriz_esparsa_tfdif.toarray(), columns = tokens)

In [ ]:
df_vocabulario = pd.DataFrame(vetorizador.vocabulary_, index = ['vocabulario']).T.sort_values(by='vocabulario', ascending = False)

In [ ]:
matriz_esparsadf

In [ ]:
df_vocabulario

In [ ]:
matriz_esparsadf.to_csv('matrizesparsa.csv',sep = ';' , encoding = 'utf-8-sig')

In [ ]:
df_limpo2b.to_csv('dflimpo.csv',sep = ';' , encoding = 'utf-8-sig')

In [ ]:
#Testando a redução de dimensionalidade com menor perda com base na variance ratio
data = matriz_esparsa_tfdif
explained =[]
n_comp = [100,150,200,500,800,900,1000,1500,2000,2500]
for x in n_comp:
    svd = TruncatedSVD(n_components = x)
    svd.fit(data)
    explained.append(svd.explained_variance_ratio_.sum())
    #lsa = NMF(n_components = x)
    #lsa.fit(data)
   # explained.append(sum(lsa.explained_variance_ratio))
    print("Número de componentes = %r e número da variância = %r"%(x,svd.explained_variance_ratio_.sum()))

In [ ]:
#Gráfico Número de features x Valor variância explicada
plt.plot(n_comp,explained)
plt.xlabel("Número de Componentes")
plt.ylabel("Variância Explicada")
plt.title("Número de Componentes x Variância Explicada")
plt.show()

In [ ]:
#Aplicando redução de dimensionalidade

t0 = time.time()
svd = TruncatedSVD(900)
lsa = make_pipeline(svd, Normalizer(copy=False))
corpus_transformed = lsa.fit_transform(matriz_esparsa_tfdif)
print("  done in %.3fsec" % (time.time() - t0))



In [ ]:
#Testando as dimensões reduzidas e a compatibiliade da matriz esparsa com as labels
print(corpus_transformed.shape)
print(len(list_labels))

In [ ]:
#Criando os modelos para fazer o treinamento

modelos =[]
modelos.append(('Árvores de Decisão', DecisionTreeClassifier()))
modelos.append(('Random Forest Classifier', RandomForestClassifier()))
modelos.append(('Gaussian Naive Bayes',GaussianNB()))
modelos.append(('Ridge Classifier', RidgeClassifier()))
modelos.append(('Linear Discriminant Analysis', LinearDiscriminantAnalysis()))
modelos.append(('Support Vector Machines', SVC()))
modelos.append(('Regressão Logística Multinomial', LogisticRegression(multi_class = 'multinomial', solver ='saga')))
modelos.append(('K Neighbors Classifier', KNeighborsClassifier()))



In [ ]:
#Criando Dicionário para os modelos
modelosDic= {'Árvores de Decisão': 'CART',
            'Random Forest Classifier': 'RFC',
            'Gaussian Naive Bayes': 'GNB',
            'Ridge Classifier': 'RIDGE',
            'Linear Discriminant Analysis': 'LDA',
            'Support Vector Machines': 'SVM',
            'Regressão Logística Multinomial': 'RLM',
            'K Neighbors Classifier': 'KNN'}
siglasmodelos = [modelosDic[x] for x in modelosDic]
print(siglasmodelos)
modelos

In [ ]:
modelos

In [ ]:
modelosDic

In [ ]:
#Criando lista para medidas de avaliação
medidasaval = ['accuracy', 'precision_weighted','recall_weighted','f1_weighted']


In [ ]:
#Criando data frame para armazenar os resultados
df_avaliacao = pd.DataFrame(columns = ['Medida', 'Algoritmo','Sigla', 'Média', 
                                       'Devio-Padrão','Coeficiente de variação', 'Valor mínimo', 'Valor Máximo'])


In [ ]:
df_avaliacao

In [ ]:
#Cross validation para avaliação de algoritmos

t0 = time.time()
for medaval in medidasaval:
    resultados =[]
    modeloslista = []

    
    for modeloNome, modelo in modelos:
        metodo_crossvalidation = model_selection.KFold (n_splits= 10, shuffle = False, random_state = None)
        scores = model_selection.cross_val_score(modelo, corpus_transformed, list_labels, cv= metodo_crossvalidation, 
                                                 scoring = medaval)
        
        resultados.append(scores)
        modeloslista.append(modeloNome)
        df_avaliacao.loc[len(df_avaliacao)]=[medaval,
                                            modeloNome,
                                            modelosDic[modeloNome],
                                            scores.mean(),
                                            scores.std(),
                                            scores.std()/scores.mean(),
                                            scores.min(),
                                            scores.max()]
        
print("  done in %.3fsec" % (time.time() - t0))
        

In [ ]:
#Visualizado o dataframe de avaliação
df_avaliacao.sort_values(['Medida', 'Média'], ascending = False) 

In [ ]:
boxplot = [modelosDic[x] for x in modeloslista]
imagem = plt.figure()
imagem.suptitle('Comparação dos Algorítimos pela acurária')
ax = imagem.add_subplot(111)
plt.boxplot(resultados)
ax.set_xticklabels(boxplot)
plt.show()

In [ ]:
 for i in resultados:
        print(i.mean())

In [ ]:
#Treinando o classificador com o melhor desempenho
x_train, x_test, y_train, y_test = train_test_split(corpus_transformed, list_labels, test_size = 0.2, random_state =0)
classificador =  RidgeClassifier(random_state = None)
classificador.fit(x_train, y_train)
print(str(classificador.score(corpus_transformed, list_labels)*100) + '%')

In [ ]:
pred = classificador.predict(x_test)

In [ ]:
#Construindo e visualizando a matriz de confusão
a = confusion_matrix(y_test, pred)
df_cm = pd.DataFrame(a, index = ['CLASSIFICAÇÃO DE MERCADORIAS',       
'CONTRIBUIÇÃO DE INTERVENÇÃO NO DOMÍNIO ECONÔMICO',       
'CONTRIBUIÇÕES SOCIAIS E PREVIDENCIÁRIAS',       
'IMPOSTO SOBRE A IMPORTAÇÃO',       
'IMPOSTO SOBRE A RENDA DE PESSOA FÍSICA',       
'IMPOSTO SOBRE A RENDA DE PESSOA JURÍDICA',       
'IMPOSTO SOBRE A RENDA RETIDO NA FONTE',       
'IMPOSTO SOBRE PRODUTOS INDUSTRIALIZADOS',       
'IOF',       
'NORMAS DE ADMINISTRAÇÃO TRIBUTÁRIA',       
'NORMAS GERAIS DE DIREITO TRIBUTÁRIO',       
'OBRIGAÇÕES ACESSÓRIAS',       
'REGIMES ADUANEIROS',       
'SIMPLES NACIONAL'],
              columns = [
    'CLASSIFICAÇÃO DE MERCADORIAS',       
'CONTRIBUIÇÃO DE INTERVENÇÃO NO DOMÍNIO ECONÔMICO',       
'CONTRIBUIÇÕES SOCIAIS E PREVIDENCIÁRIAS',       
'IMPOSTO SOBRE A IMPORTAÇÃO',       
'IMPOSTO SOBRE A RENDA DE PESSOA FÍSICA',       
'IMPOSTO SOBRE A RENDA DE PESSOA JURÍDICA',       
'IMPOSTO SOBRE A RENDA RETIDO NA FONTE',       
'IMPOSTO SOBRE PRODUTOS INDUSTRIALIZADOS',       
'IOF',       
'NORMAS DE ADMINISTRAÇÃO TRIBUTÁRIA',       
'NORMAS GERAIS DE DIREITO TRIBUTÁRIO',       
'OBRIGAÇÕES ACESSÓRIAS',       
'REGIMES ADUANEIROS',       
'SIMPLES NACIONAL'])
plt.figure(figsize = (20,10))
sn.heatmap(df_cm, annot=True,cmap="OrRd", vmin=0, vmax=15)

In [ ]:
#Relatório de Classificação
print('\nRelatório de classificação: \n', classification_report(y_test, pred))
